# Functions for maps

Topography

In [29]:
def get_Dem(huc,ras,project_dir):
    layout_name = "DEM"
    def getlabel(k,lbl):
        parts = b.split(" - ")
        if k==0:
            rounded= round(float(parts[1]))
            brk_label= "Below "+str(rounded)
        elif k==9:
            rounded= round(float(parts[0]))
            brk_label= "Above "+str(rounded)
        else:
            # Round and convert each part to a float
            rounded_numbers = [round(float(part)) for part in parts]
            # Convert the rounded floats back to strings
            rounded_strings = [f"{rounded}" for rounded in rounded_numbers]
            # Create a new formatted string
            brk_label = " - ".join(rounded_strings)
        return brk_label
    
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''

    
    
    #ras_file=dem
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name=os.path.split(ras)
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]

    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary=layers_in_layout[0]
    raster=layers_in_layout[1]
    boundary_src=layers_in_layout[0].connectionProperties
    raster_src=layers_in_layout[1].connectionProperties
    new_conn_v = {'dataset':file_name,'workspace_factory': 'File Geodatabase','connection_info':{'database': directory}}
    new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    sym = raster.symbology
    sym.colorizer.breakCount = 10
    sym.colorizer.classificationField="Feet,NAVD88"
    # Round up the upper bounds of class breaks to two decimal places and update labels
    for j in range(sym.colorizer.breakCount):
        upper_bound = round(sym.colorizer.classBreaks[j].upperBound)
        sym.colorizer.classBreaks[j].upperBound = upper_bound
        #print(sym.colorizer.classBreaks[i].label)
        b=sym.colorizer.classBreaks[j].label
        brk_label=getlabel(j,b)
        sym.colorizer.classBreaks[j].label = brk_label
        #print(sym.colorizer.classBreaks[i].label)
    raster.symbology = sym
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    #original_extent = boundary.extent
    #mf.camera.setExtent(original_extent)
    # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            # Call a function that uses huc and hucid
            # Assuming the function name is get_name and returns a name string
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name=='':
                # Rename the layers
                boundary.name = hucid
            else:
                boundary.name = "HUC "+ huc_name +" "+str(hucid)
            # Define the definition query
            definition_query = f"{id_col} = '{hucid}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        # delete definition query to the layer
        boundary.definitionQuery = ''

    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    # Clean up by releasing the project
    del aprx
    return map_file


Water Table Elevation

In [30]:
def get_Wte(huc,ras,project_dir):
    layout_name = "WTE"
    def getlabel(k,lbl):
        parts = b.split(" - ")
        if k==0:
            rounded= round(float(parts[1]))
            brk_label= "Below "+str(rounded)
        elif k==5:
            rounded= round(float(parts[0]))
            brk_label= "Above "+str(rounded)
        else:
            # Round and convert each part to a float
            rounded_numbers = [round(float(part)) for part in parts]
            # Convert the rounded floats back to strings
            rounded_strings = [f"{rounded}" for rounded in rounded_numbers]
            # Create a new formatted string
            brk_label = " - ".join(rounded_strings)
        return brk_label
    
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''

    
    #ras_file=dem
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name=os.path.split(ras)
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]
    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary=layers_in_layout[0]
    raster=layers_in_layout[1]
    boundary_src=layers_in_layout[0].connectionProperties
    raster_src=layers_in_layout[1].connectionProperties
    new_conn_v = {'dataset':file_name,'workspace_factory': 'File Geodatabase','connection_info':{'database': directory}}
    new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    sym = raster.symbology
    sym.colorizer.breakCount = 6
    sym.colorizer.classificationField="Feet,NAVD88"
    # Round up the upper bounds of class breaks to two decimal places and update labels
    for j in range(sym.colorizer.breakCount):
        upper_bound = round(sym.colorizer.classBreaks[j].upperBound)
        sym.colorizer.classBreaks[j].upperBound = upper_bound
        #print(sym.colorizer.classBreaks[i].label)
        b=sym.colorizer.classBreaks[j].label
        brk_label=getlabel(j,b)
        sym.colorizer.classBreaks[j].label = brk_label
        #print(sym.colorizer.classBreaks[i].label)
    raster.symbology = sym
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    #original_extent = boundary.extent
    #mf.camera.setExtent(original_extent)
    # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            # Call a function that uses huc and hucid
            # Assuming the function name is get_name and returns a name string
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name=='':
                # Rename the layers
                boundary.name = hucid
            else:
                boundary.name = "HUC "+ huc_name +" "+str(hucid)
            # Define the definition query
            definition_query = f"{id_col} = '{hucid}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        # delete definition query to the layer
        boundary.definitionQuery = ''
        
    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)

    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    # Clean up by releasing the project
    del aprx
    return map_file
            

Impervious

In [31]:
def get_Impervious(huc,ras,project_dir):
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''

    # Access the layout 
    layout_name = "Impervious"
    #ras_file=dem
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name=os.path.split(ras)
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]
    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary=layers_in_layout[0]
    raster=layers_in_layout[1]
    boundary_src=layers_in_layout[0].connectionProperties
    raster_src=layers_in_layout[1].connectionProperties
    new_conn_v = {'dataset':file_name,'workspace_factory': 'File Geodatabase','connection_info':{'database': directory}}
    new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    #original_extent = boundary.extent
    #mf.camera.setExtent(original_extent)
    # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            # Call a function that uses huc and hucid
            # Assuming the function name is get_name and returns a name string
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name=='':
                # Rename the layers
                boundary.name = hucid
            else:
                boundary.name = "HUC "+ huc_name +" "+str(hucid)
            # Define the definition query
            definition_query = f"{id_col} = '{hucid}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        # delete definition query to the layer
        boundary.definitionQuery = ''
        
    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    # Clean up by releasing the project
    del aprx
    return map_file

Water Binary

In [32]:
def get_Water(huc,ras,outdir):
    
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''

    # Access the layout 
    layout_name = "Water"
    #ras_file=dem
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name=os.path.split(ras)
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]
    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary=layers_in_layout[0]
    raster=layers_in_layout[1]
    boundary_src=layers_in_layout[0].connectionProperties
    raster_src=layers_in_layout[1].connectionProperties
    new_conn_v = {'dataset':file_name,'workspace_factory': 'File Geodatabase','connection_info':{'database': directory}}
    new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    #original_extent = boundary.extent
    #mf.camera.setExtent(original_extent)
    # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            # Call a function that uses huc and hucid
            # Assuming the function name is get_name and returns a name string
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name=='':
                # Rename the layers
                boundary.name = hucid
            else:
                boundary.name = "HUC "+ huc_name +" "+str(hucid)
            # Define the definition query
            definition_query = f"{id_col} = '{hucid}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        # delete definition query to the layer
        boundary.definitionQuery = ''
        
    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    # Clean up by releasing the project
    del aprx
    return map_file

Rainfall

In [33]:
import os
import re
import arcpy

def get_Rain(huc, ras, outdir):
    
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''

    # Access the layout 
    layout_name = "Rainfall"
    
    def get_names(fname):
        # Define the pattern to match both "1d100y" and "100y1d"
        pattern = r'_(\d+)y(\d+)d|_(\d+)d(\d+)y'

        # Search for the pattern in the filename
        match = re.search(pattern, fname)

        if match:
            if match.group(1) and match.group(2):
                year, day = match.group(1), match.group(2)
            else:
                year, day = match.group(4), match.group(3)
            
            lyrname = f'{day}-Day {year}-Year Rainfall'
            title = f'Rainfall, {day}-Day {year}-Year \nStorm Event'
            
            return lyrname, title
        else:
            return 'Null', 'Null'
    
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name = os.path.split(ras)
    lyrname, title = get_names(ras_name)  # Call the function to get lyrname and title
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)
    # Access the layout named "DEM"
    layout = aprx.listLayouts(layout_name)[0]
    title_element = layout.listElements("TEXT_ELEMENT")[1]

    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary = layers_in_layout[0]
    raster = layers_in_layout[1]
    boundary_src = layers_in_layout[0].connectionProperties
    raster_src = layers_in_layout[1].connectionProperties
    
    new_conn_v = {'dataset': file_name, 'workspace_factory': 'File Geodatabase', 'connection_info': {'database': directory}}
    new_conn_r = {'dataset': ras_name, 'workspace_factory': 'Raster', 'connection_info': {'database': ras_folder}}
    
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src, new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    
    sym = raster.symbology
    sym.colorizer.breakCount = 10
    sym.colorizer.classificationField = "Inches"
    
    # Round up the upper bounds of class breaks to two decimal places and update labels
    for j in range(sym.colorizer.breakCount):
        upper_bound = round(sym.colorizer.classBreaks[j].upperBound, 2)
        sym.colorizer.classBreaks[j].upperBound = upper_bound
        b = sym.colorizer.classBreaks[j].label
        parts = b.split(" - ")
        rounded_numbers = [round(float(part), 2) for part in parts]
        rounded_strings = [f"{rounded:.2f}" for rounded in rounded_numbers]
        brk_label = " - ".join(rounded_strings)
        sym.colorizer.classBreaks[j].label = brk_label

    raster.symbology = sym
    
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    
    raster.name = lyrname
    title_element.text = title

    # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name == '':
                boundary.name = hucid
            else:
                boundary.name = "HUC " + huc_name + " " + str(hucid)
            definition_query = f"{id_col} = '{hucid}'"
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        boundary.definitionQuery = ''
        
    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)
    
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    
    # Clean up by releasing the project
    del aprx
    
    return map_file


WHC Ratio

In [34]:
def get_Whc(huc, ras, outdir):
    
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''

    # Access the layout 
    layout_name = "WHC_Ratio"
    
    classbrks=[0,0.05,0.10,0.15,0.20,0.25,1]
    labels=['0,No Capacity','0.00-0.05','0.05-0.10','0.10-0.15','0.15-0.20','0.20-0.25','0.25-1.00']

    directory, file_name = os.path.split(huc)
    ras_folder, ras_name = os.path.split(ras)
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]

    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary=layers_in_layout[0]
    raster=layers_in_layout[1]
    boundary_src=layers_in_layout[0].connectionProperties
    raster_src=layers_in_layout[1].connectionProperties
    new_conn_v = {'dataset':file_name,'workspace_factory': 'File Geodatabase','connection_info':{'database': directory}}
    new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    sym = raster.symbology
    sym.colorizer.classificationField="Ratio"
    sym.colorizer.breakCount = len(classbrks)
    breaks=sym.colorizer.classBreaks
    i=0
    for brk in breaks:
        brk.upperBound = classbrks[i]
        brk.label =labels[i]
        i=i+1
    raster.symbology = sym
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    
     # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name == '':
                boundary.name = hucid
            else:
                boundary.name = "HUC " + huc_name + " " + str(hucid)
            definition_query = f"{id_col} = '{hucid}'"
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        boundary.definitionQuery = ''
        
    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)
    
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    
    # Clean up by releasing the project
    del aprx
    
    return map_file

Unsaturated Zone

In [35]:
def get_Unsaturated(huc,ras,outdir):
    
    layout_name = "UnsaturatedZone"
    
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''
    
    def getlabel(k,lbl):
        parts = b.split(" - ")
        if k==0:
            rounded= round(float(parts[1]))
            brk_label= "Below "+str(rounded)
        elif k==5:
            rounded= round(float(parts[0]))
            brk_label= "Above "+str(rounded)
        else:
            # Round and convert each part to a float
            rounded_numbers = [round(float(part)) for part in parts]
            # Convert the rounded floats back to strings
            rounded_strings = [f"{rounded}" for rounded in rounded_numbers]
            # Create a new formatted string
            brk_label = " - ".join(rounded_strings)
        return brk_label
    #ras_file=dem
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name=os.path.split(ras)
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]
    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary=layers_in_layout[0]
    raster=layers_in_layout[1]
    boundary_src=layers_in_layout[0].connectionProperties
    raster_src=layers_in_layout[1].connectionProperties
    new_conn_v = {'dataset':file_name,'workspace_factory': 'File Geodatabase','connection_info':{'database': directory}}
    new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    sym = raster.symbology
    sym.colorizer.breakCount = 6
    sym.colorizer.classificationField="Depth in Feet"
    # Round up the upper bounds of class breaks to two decimal places and update labels
    for j in range(sym.colorizer.breakCount):
        upper_bound = round(sym.colorizer.classBreaks[j].upperBound)
        sym.colorizer.classBreaks[j].upperBound = upper_bound
        #print(sym.colorizer.classBreaks[i].label)
        b=sym.colorizer.classBreaks[j].label
        brk_label=getlabel(j,b)
        sym.colorizer.classBreaks[j].label = brk_label
        #print(sym.colorizer.classBreaks[i].label)
    raster.symbology = sym
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    #original_extent = boundary.extent
    #mf.camera.setExtent(original_extent)
     # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name == '':
                boundary.name = hucid
            else:
                boundary.name = "HUC " + huc_name + " " + str(hucid)
            definition_query = f"{id_col} = '{hucid}'"
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        boundary.definitionQuery = ''
        
    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    # Clean up by releasing the project
    del aprx
    return map_file

In [36]:
def get_D2Wte(huc,ras,outdir):
    layout_name = "UnsaturatedZone"
    
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''
    
    def getlabel(k,lbl):
        parts = b.split(" - ")
        if k==0:
            rounded= round(float(parts[1]))
            brk_label= "Below "+str(rounded)
        elif k==5:
            rounded= round(float(parts[0]))
            brk_label= "Above "+str(rounded)
        else:
            # Round and convert each part to a float
            rounded_numbers = [round(float(part)) for part in parts]
            # Convert the rounded floats back to strings
            rounded_strings = [f"{rounded}" for rounded in rounded_numbers]
            # Create a new formatted string
            brk_label = " - ".join(rounded_strings)
        return brk_label
    #ras_file=dem
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name=os.path.split(ras)
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]
    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary=layers_in_layout[0]
    raster=layers_in_layout[1]
    boundary_src=layers_in_layout[0].connectionProperties
    raster_src=layers_in_layout[1].connectionProperties
    new_conn_v = {'dataset':file_name,'workspace_factory': 'File Geodatabase','connection_info':{'database': directory}}
    new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    sym = raster.symbology
    sym.colorizer.breakCount = 6
    sym.colorizer.classificationField="Depth in Feet"
    # Round up the upper bounds of class breaks to two decimal places and update labels
    for j in range(sym.colorizer.breakCount):
        upper_bound = round(sym.colorizer.classBreaks[j].upperBound)
        sym.colorizer.classBreaks[j].upperBound = upper_bound
        #print(sym.colorizer.classBreaks[i].label)
        b=sym.colorizer.classBreaks[j].label
        brk_label=getlabel(j,b)
        sym.colorizer.classBreaks[j].label = brk_label
        #print(sym.colorizer.classBreaks[i].label)
    raster.symbology = sym
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    #original_extent = boundary.extent
    #mf.camera.setExtent(original_extent)
    # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name == '':
                boundary.name = hucid
            else:
                boundary.name = "HUC " + huc_name + " " + str(hucid)
            definition_query = f"{id_col} = '{hucid}'"
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        boundary.definitionQuery = ''
        
    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    # Clean up by releasing the project
    del aprx
    return map_file

Soil Storage

In [37]:
def get_Ssc(huc,ras,outdir):
    
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''
    
    layout_name = "SoilStorage"
    def getlabel(k,lbl):
        parts = b.split(" - ")
        if k==0:
            rounded= round(float(parts[1]))
            brk_label= "Below "+str(rounded)
        elif k==5:
            rounded= round(float(parts[0]))
            brk_label= "Above "+str(rounded)
        else:
            # Round and convert each part to a float
            rounded_numbers = [round(float(part)) for part in parts]
            # Convert the rounded floats back to strings
            rounded_strings = [f"{rounded}" for rounded in rounded_numbers]
            # Create a new formatted string
            brk_label = " - ".join(rounded_strings)
        return brk_label
    #ras_file=dem
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name=os.path.split(ras)
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]
    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary=layers_in_layout[0]
    raster=layers_in_layout[1]
    boundary_src=layers_in_layout[0].connectionProperties
    raster_src=layers_in_layout[1].connectionProperties
    new_conn_v = {'dataset':file_name,'workspace_factory':  'File Geodatabase','connection_info':{'database': directory}}
    new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    sym = raster.symbology
    sym.colorizer.breakCount = 6
    sym.colorizer.classificationField="Depth in Feet"
    # Round up the upper bounds of class breaks to two decimal places and update labels
    for j in range(sym.colorizer.breakCount):
        upper_bound = round(sym.colorizer.classBreaks[j].upperBound)
        sym.colorizer.classBreaks[j].upperBound = upper_bound
        #print(sym.colorizer.classBreaks[i].label)
        b=sym.colorizer.classBreaks[j].label
        brk_label=getlabel(j,b)
        sym.colorizer.classBreaks[j].label = brk_label
        #print(sym.colorizer.classBreaks[i].label)
    raster.symbology = sym
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    #original_extent = boundary.extent
    #mf.camera.setExtent(original_extent)
    # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name == '':
                boundary.name = hucid
            else:
                boundary.name = "HUC " + huc_name + " " + str(hucid)
            definition_query = f"{id_col} = '{hucid}'"
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        boundary.definitionQuery = ''
        
    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    # Clean up by releasing the project
    del aprx
    return map_file

NLCD Landcover

In [38]:
def get_Nlcd(huc,ras,outdir):
    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''
    
    # Access the layout 
    layout_name = "NLCD_LandCover"
    #ras_file=dem
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name=os.path.split(ras)
    print(ras_name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]
    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary=layers_in_layout[0]
    raster=layers_in_layout[1]
    boundary_src=layers_in_layout[0].connectionProperties
    raster_src=layers_in_layout[1].connectionProperties
    new_conn_v = {'dataset':file_name,'workspace_factory': 'File Geodatabase','connection_info':{'database': directory}}
    new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
    # Update the data source for layers_in_layout[0]
    layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    #original_extent = boundary.extent
    #mf.camera.setExtent(original_extent)
    #boundary.name = "HUC12s Boundary"
    sym = raster.symbology
    sym.colorizer.classificationField="NLCD (2016) Classification"

    # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)
        
    # delete definition query to the layer
    boundary.definitionQuery = ''

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name == '':
                boundary.name = hucid
            else:
                boundary.name = "HUC " + huc_name + " " + str(hucid)
            definition_query = f"{id_col} = '{hucid}'"
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        boundary.definitionQuery = ''
        
    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()
    # Clean up by releasing the project
    del aprx
    return map_file


Inundation

In [39]:
import os
import re
import arcpy

def get_Inundation(huc, ras, project_dir):
    def lname(strng):
        scenario = strng
        patterns = [r'(\d+d\d+yr)', r'(\d+ft)', r'(kt)', r'(pump)']
        matches = [re.findall(pattern, scenario) for pattern in patterns]

        if matches[0]:
            d, y = matches[0][0].split('d')
        substring1 = f"{d}-day {y.replace('yr', '-year ')}storm event" if matches[0] else ''
        substring2 = f"{matches[1][0].replace('ft', '')}-feet sea level rise" if matches[1] else ''
        substring3 = '2.6ft King Tides' if matches[2] else ''
        substring4 = 'after installing pumps at outlets' if matches[3] else ''

        if substring1 == substring2 == substring4 == '' and substring3:
            lname = f'Probability of inundation based on {substring3}'
        elif substring4 == substring3 == substring2 == '' and substring1:
            lname = f'Probability of inundation based on the {substring1}'
        elif substring1 and substring2 and not (substring3 or substring4):
            lname = f'Probability of inundation based on {substring2} and {substring1}'
        elif substring4:
            lname = f'Probability of inundation based on {substring2} and {substring1} {substring4}'
        elif substring1 and not substring4 and not substring2 and substring3:
            lname = f'Probability of inundation based on {substring3} and {substring1}'
        else:
            lname = f'Probability of inundation based on {substring3},{substring2} and {substring1}'

        return lname

    def round_scale(number):
        return (int(number / 1000) * 1000) + (2000 if number < 100000 else 5000)

    def get_huc_name(shp_file, huc):
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", huc_col, id_col]) as cursor:
            for row in cursor:
                print(row[2])
                if str(row[2]) == str(huc):
                    return row[1]
        return ''

    # Extract directory and filename from paths
    directory, file_name = os.path.split(huc)
    ras_folder, ras_name = os.path.split(ras)
    scenario = ras_name
    print(ras_name)

    # Define the path to your ArcGIS Pro project file (.aprx)
    project_path = "current"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject(project_path)

    # Access the layout named "Zscore_FloodRisk"
    layout_name = "Zscore_FloodRisk"
    layout = aprx.listLayouts(layout_name)[0]

    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    boundary, smlyr, water, flood = layers_in_layout[0], layers_in_layout[2], layers_in_layout[3], layers_in_layout[4]

    # Update the data source for boundary, water, and flood layers
    new_conn_v = {'dataset': file_name, 'workspace_factory': 'File Geodatabase', 'connection_info': {'database': directory}}
    new_conn_r = {'dataset': 'water.tif', 'workspace_factory': 'Raster', 'connection_info': {'database': directory}}
    new_conn_f = {'dataset': ras_name, 'workspace_factory': 'Raster', 'connection_info': {'database': ras_folder}}

    boundary.updateConnectionProperties(boundary.connectionProperties, new_conn_v)
    water.updateConnectionProperties(water.connectionProperties, new_conn_r)
    flood.updateConnectionProperties(flood.connectionProperties, new_conn_f)

    # Update the layer name for smlyr
    smlyr.name = lname(scenario)

    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.camera.setExtent(mf.getLayerExtent(flood, False, True))

    # Define the output directory for maps
    maps_dir = ras_folder.replace(project_dir, os.path.join(project_dir, 'Maps'))

    # Ensure the output folder exists, or create it if it doesn't
    if not os.path.exists(maps_dir):
        os.makedirs(maps_dir)

    # Extract hucid from raster path if "HUC12s" is present
    hucid = ''
    if 'HUC12s' in ras_folder:
        match = re.search(r'HUC12s\\(\d{12})', ras_folder)
        if match:
            hucid = match.group(1)
            # Call a function that uses huc and hucid
            # Assuming the function name is get_name and returns a name string
            huc_name = get_huc_name(huc, hucid)
            print(f"Extracted Name: {huc_name}")
            if huc_name=='':
                # Rename the layers
                boundary.name = hucid
            else:
                boundary.name = "HUC "+ huc_name +" "+str(hucid)
            # Define the definition query
            definition_query = f"{id_col} = '{hucid}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
    else:
        boundary.name = "HUC12s Boundary"
        # delete definition query to the layer
        boundary.definitionQuery = ''

    # Define map file name and path
    map_name = os.path.splitext(ras_name)[0] + '.jpg'
    map_file = os.path.join(maps_dir, map_name)

    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    
    # Save the project
    aprx.save()

    # Clean up by releasing the project
    del aprx
    return map_file




### Section 4.1 – Automated Map Creation for Flood Scenario Outputs

This section automates the generation of map documents using predefined rendering functions for each raster output. It scans through processed raster datasets and applies the appropriate symbology and layout settings to generate scenario-specific flood maps.

---

#### 🎯 Purpose

Each raster layer (e.g., `dem_ft`, `soilStorage`, `unsaturated`, `r1d100yr`, `SCC3ft`) is rendered using a dedicated function that handles the layout, labeling, symbology, and export logic. This ensures consistent map outputs across datasets and scenarios.

---

#### 🗂 Folder Configuration

The script is designed to work with the following directory structure:

```
Project_Directory/
├── HUC12s/
├── mosaic/
├── Homestead.gdb/
└── (Map output folders as created by rendering functions)
```

You should set:

```python
project_dir = r"X:\amandal2023$\Working\Homestead\Homestead_City"
```

- `huc12s_dir`: Contains HUC subdirectories with raster outputs.
- `mosaic_dir`: Contains mosaicked raster datasets.
- `gdb_name`, `huc_fname`, `huc_col`, `id_col`: Refer to the geodatabase and HUC boundary settings.

---

#### ⚙️ How the Script Works

1. **File Search**
   - The script recursively scans the `HUC12s/` and `mosaic/` directories for raster files with `.tif`, `.dat`, or `.img` extensions.

2. **Prefix Detection**
   - Each raster filename is parsed to extract a prefix (e.g., `"dem_ft"` → `Dem` → `get_Dem`).
   - The script checks if a function named `get_{Prefix}` exists.

3. **Function Execution**
   - If a matching function exists (e.g., `get_SoilStorage`, `get_Zscore`), it is called with:
     - `huc_path`: Path to the HUC boundary feature class
     - `file`: Path to the raster file
     - `project_dir`: Root project directory

4. **Map Generation**
   - Inside each function:
     - A map layout is created in ArcGIS Pro
     - The raster is added and styled
     - Text elements and legends are updated
     - The layout is exported as a PDF or image to the specified location

5. **Output Handling**
   - If the function returns a path, it is printed
   - Otherwise, a warning is displayed indicating the map was not generated

---

#### ✅ Requirements

- All map-rendering functions (e.g., `get_Dem`, `get_SoilStorage`, `get_Zscore`) must be defined in the same script or imported module.
- Raster filenames must begin with a prefix that matches one of the defined functions.
- Ensure ArcGIS Pro Python environment has access to arcpy and the Spatial Analyst extension.

---

#### 📤 Output

Each function outputs one map layout per raster. These maps are typically exported as:

- `.pdf` or `.png` files
- Named and stored in scenario-specific folders (e.g., `/Maps/r1d100yr/`, `/Maps/SCC3ft/`)

---

#### 📝 Final Notes

- This approach streamlines map production for large scenario sets.
- Naming conventions are essential for automation to work.
- Add new rendering functions for additional raster types by using the naming pattern: `get_{Prefix}`.


In [40]:
%%time
import os
import glob
import arcpy
import re
from arcpy.sa import *
import time


arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

# Define the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'
huc12s_dir = os.path.join(project_dir, 'HUC12s')
mosaic_dir = os.path.join(project_dir, 'mosaic')
extensions = ['.tif', '.dat', '.img']  # List of extensions to search for
# Settings for the HUC boundary
gdb_name = "Homestead"
huc_fname = "HUC12s_Boundary"  # Name of the feature class
huc_col = "NAME"  # Column representing the name of the HUCs
id_col = "HUC12"  # Column representing the ID of the HUCs
# Path to the geodatabase
gdb_path = os.path.join(project_dir, f"{gdb_name}.gdb")
huc_path = os.path.join(gdb_path, huc_fname)


# Function to list files with specific extensions from a directory and its subdirectories
def list_files(directory, extensions):
    files = []
    for ext in extensions:
        # Use glob to search for files in subdirectories
        files.extend(glob.glob(os.path.join(directory, '**', f'*{ext}'), recursive=True))
    return files

# List all files in the mosaic directory and HUC12s directory (including subdirectories)
directories = [mosaic_dir, huc12s_dir]
files = []
for directory in directories:
    files.extend(list_files(directory, extensions))

# Process each file
for file in files:
    fname = os.path.basename(file)
    print(file)
    # Split the filename and extension
    file_parts = fname.split('.')[0]
    # Extract prefix based on known patterns (e.g., first part before underscore)
    prefix = file_parts.split('_')[0].title()
    fun_name = f'get_{prefix}'
    if fun_name in globals() and callable(globals()[fun_name]):
        get_map = globals()[fun_name](huc_path, file, project_dir)
        if get_map:
            print(get_map)
        else:
            print('Map has not been generated')
    else:
        print(f"No function found for prefix: {prefix}")

print("Processing completed.")


X:\amandal2023$\Working\Homestead\Homestead_City\mosaic\dem_ft.tif
dem_ft.tif
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\mosaic\dem_ft.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\mosaic\impervious.tif
impervious.tif
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\mosaic\impervious.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\mosaic\Inundation_1d100yr.tif
Inundation_1d100yr.tif
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\mosaic\Inundation_1d100yr.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\mosaic\Inundation_1d100yr_1ft.tif
Inundation_1d100yr_1ft.tif
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\mosaic\Inundation_1d100yr_1ft.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\mosaic\Inundation_1d100yr_1ft_kt.tif
Inundation_1d100yr_1ft_kt.tif
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\mosaic\Inundation_1d100yr_1ft_kt.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\mosaic\Inundation_1d100yr_2ft.tif
Inundation_1d1

Inundation_1d10yr_1ft_kt.tif
030902061607
030902061605
Extracted Name: MOWERY CANAL
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061605_MoweryCanal\Inundation_1d10yr_1ft_kt.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\Inundation_1d10yr_2ft.tif
Inundation_1d10yr_2ft.tif
030902061607
030902061605
Extracted Name: MOWERY CANAL
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061605_MoweryCanal\Inundation_1d10yr_2ft.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\Inundation_1d10yr_2ft_kt.tif
Inundation_1d10yr_2ft_kt.tif
030902061607
030902061605
Extracted Name: MOWERY CANAL
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061605_MoweryCanal\Inundation_1d10yr_2ft_kt.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\Inundation_1d10yr_3ft.tif
Inundation_1d10yr_3ft.tif
030902061607
030902061605
Extracted Name: MOWERY CANAL
X:\a

X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061606_CanalS177\Inundation_1d100yr_5ft.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\Inundation_1d100yr_5ft_kt.tif
Inundation_1d100yr_5ft_kt.tif
030902061607
030902061605
030902061606
Extracted Name: CANAL S177
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061606_CanalS177\Inundation_1d100yr_5ft_kt.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\Inundation_1d100yr_kt.tif
Inundation_1d100yr_kt.tif
030902061607
030902061605
030902061606
Extracted Name: CANAL S177
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061606_CanalS177\Inundation_1d100yr_kt.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\Inundation_1d10yr.tif
Inundation_1d10yr.tif
030902061607
030902061605
030902061606
Extracted Name: CANAL S177
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061606_

Inundation_1d100yr_2ft_kt.tif
030902061607
Extracted Name: MODEL LAND CANAL
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061607_ModelLandCanal\Inundation_1d100yr_2ft_kt.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\Inundation_1d100yr_3ft.tif
Inundation_1d100yr_3ft.tif
030902061607
Extracted Name: MODEL LAND CANAL
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061607_ModelLandCanal\Inundation_1d100yr_3ft.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\Inundation_1d100yr_3ft_kt.tif
Inundation_1d100yr_3ft_kt.tif
030902061607
Extracted Name: MODEL LAND CANAL
X:\amandal2023$\Working\Homestead\Homestead_City\Maps\HUC12s\030902061607_ModelLandCanal\Inundation_1d100yr_3ft_kt.jpg
X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\Inundation_1d100yr_4ft.tif
Inundation_1d100yr_4ft.tif
030902061607
Extracted Name: MODEL LAND CANAL
X:\amandal20